You are an expert Python developer specializing in the Databricks environment. Your task is to create a complete Python script to be executed within a Databricks notebook. The script must perform the following operations:
1.	Data Retrieval from SpaceX API:
o	Interact with the SpaceX v3 REST API (https://api.spacexdata.com/v3).
o	Retrieve data from one specific endpoint: 
	All launches: https://api.spacexdata.com/v3/launches
o	Handle potential errors during the API calls (e.g., timeouts, non-200 status codes).
2.	Filter Operation:
o	Perform a "filter" operation on the retrieved launch data based on specific criteria.
o	Filter Logic: Filter the list of launches based on launch year and launch success status. Only include launches that match the specified criteria.
o	The result should be a list of dictionaries, containing only the launch records that satisfy the filter conditions.
3.	Control Parameters and Debugging:
o	Include variables at the beginning of the script to define the API endpoint URL and the filtering criteria, making them easily modifiable. Use parameters like: 
	API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
	FILTER_YEAR = 2019 (Set to an integer year, or None to disable year filtering)
	FILTER_SUCCESS = True (Set to True, False, or None to disable success filtering)
o	Use Python's standard logging module to provide informative output during execution. Configure logging to display messages at the INFO level.
o	Log key messages such as: starting data retrieval, number of launches retrieved, filtering criteria being applied, number of launches before filtering, number of launches after filtering, starting upload to httpbin, upload outcome.
4.	Execution Time Measurement:
o	Code Execution Time: Measure the time taken to perform the main operations (data retrieval + filtering). Print this time after the filtering operation is complete.
o	Pipeline Execution Time: Measure the total execution time of the entire script (from the beginning until after the upload to httpbin). Print this total time at the end of the script. Use Python's time module.
5.	Upload Result:
o	Take the resulting filtered list from the filter operation.
o	Serialize it into JSON format.
o	Make an HTTP POST request to the https://httpbin.org/post endpoint, sending the resulting filtered JSON data in the request body.
o	Verify the response from httpbin.org (e.g., check the status code) and log the outcome of the upload operation.


In [0]:
# SpaceX Data Pipeline
# This script retrieves launch data from the SpaceX API, filters it based on 
# specified criteria, and uploads the filtered data to httpbin.org

import requests
import json
import logging
import time
from datetime import datetime

# Control Parameters - Easily modifiable
API_ENDPOINT_URL = "https://api.spacexdata.com/v3/launches"
FILTER_YEAR = 2019  # Set to an integer year, or None to disable year filtering
FILTER_SUCCESS = True  # Set to True, False, or None to disable success filtering
UPLOAD_URL = "https://httpbin.org/post"

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()

def get_spacex_launches():
    """
    Retrieves launch data from SpaceX API with error handling
    """
    logger.info(f"Starting data retrieval from {API_ENDPOINT_URL}")
    
    try:
        response = requests.get(API_ENDPOINT_URL, timeout=30)
        response.raise_for_status()  # Raise exception for non-200 status codes
        launches = response.json()
        logger.info(f"Successfully retrieved {len(launches)} launches from SpaceX API")
        return launches
    except requests.exceptions.Timeout:
        logger.error("Request to SpaceX API timed out")
        raise
    except requests.exceptions.HTTPError as e:
        logger.error(f"HTTP error occurred: {e}")
        raise
    except requests.exceptions.RequestException as e:
        logger.error(f"Request error occurred: {e}")
        raise
    except json.JSONDecodeError:
        logger.error("Failed to parse JSON response")
        raise

def filter_launches(launches, year=None, success=None):
    """
    Filters launches based on year and success status
    
    Args:
        launches (list): List of launch dictionaries
        year (int, optional): Filter by launch year
        success (bool, optional): Filter by launch success status
    
    Returns:
        list: Filtered list of launch dictionaries
    """
    filter_criteria = []
    
    if year is not None:
        filter_criteria.append(f"year = {year}")
    if success is not None:
        filter_criteria.append(f"success = {success}")
    
    filter_description = " and ".join(filter_criteria)
    logger.info(f"Applying filter: {filter_description}")
    logger.info(f"Number of launches before filtering: {len(launches)}")
    
    filtered_launches = launches
    
    if year is not None:
        filtered_launches = [
            launch for launch in filtered_launches 
            if datetime.utcfromtimestamp(launch['launch_date_unix']).year == year
        ]
    
    if success is not None:
        filtered_launches = [
            launch for launch in filtered_launches 
            if launch['launch_success'] == success
        ]
    
    logger.info(f"Number of launches after filtering: {len(filtered_launches)}")
    return filtered_launches

def upload_to_httpbin(data):
    """
    Uploads JSON data to httpbin.org and verifies the response
    
    Args:
        data (list): Data to be uploaded
    
    Returns:
        bool: Success status of the upload
    """
    logger.info(f"Starting upload to {UPLOAD_URL}")
    
    try:
        json_data = json.dumps(data)
        headers = {'Content-Type': 'application/json'}
        response = requests.post(UPLOAD_URL, data=json_data, headers=headers, timeout=30)
        response.raise_for_status()
        
        # Verify the response contains our data
        response_data = response.json()
        if response_data and 'data' in response_data:
            logger.info(f"Upload successful. Server responded with status code: {response.status_code}")
            return True
        else:
            logger.warning("Upload response did not contain expected data structure")
            return False
    except Exception as e:
        logger.error(f"Upload failed: {str(e)}")
        return False

# Main execution
def main():
    # Start measuring total execution time
    total_start_time = time.time()
    
    try:
        # Data retrieval and filtering with timing
        operation_start_time = time.time()
        launches = get_spacex_launches()
        filtered_launches = filter_launches(launches, year=FILTER_YEAR, success=FILTER_SUCCESS)
        operation_end_time = time.time()
        
        # Calculate and log operation time
        operation_time = operation_end_time - operation_start_time
        logger.info(f"Data retrieval and filtering completed in {operation_time:.2f} seconds")
        
        # Upload results to httpbin
        upload_success = upload_to_httpbin(filtered_launches)
        
        # Log upload outcome
        if upload_success:
            logger.info("Data pipeline completed successfully")
        else:
            logger.warning("Data pipeline completed with warnings")
        
    except Exception as e:
        logger.error(f"Data pipeline failed: {str(e)}")
    
    # Calculate and log total execution time
    total_end_time = time.time()
    total_time = total_end_time - total_start_time
    logger.info(f"Total pipeline execution time: {total_time:.2f} seconds")

# Execute the main function
if __name__ == "__main__":
    main()